In [12]:
#using xml-sites to store all cafe-links in a list
# https://europeancoffeetrip.com/cafe-sitemap.xml bis https://europeancoffeetrip.com/cafe-sitemap5.xml

#iterating through the list -> request every website and scrape the following values -> Title, Address, Coordinates (later opening hours etc)
    #coordinates -> <div id="cafe-map">
#using rotating proxies to not get blocked
#storing the values in a dict(cafelink:[name,address,coordinates])

#the next step is also in the seperate notebook -> CafeDictionaryTOgeojson.ipynb
#creating the geojson format to import into umap.openstreetmap ->values in geojson name:titel description:address coordinates:[x,y]

In [13]:
#Method to request the Website, return value is the html-file of the website
import requests

def getHMTLfile_By_Link(link:str):
    response = requests.get(link)
    if response.status_code == 200:
        print(f"Request to {link} was successful")
        return response.text
    else:
        print(f"Request to {link} failed")
        return None

In [14]:
from bs4 import BeautifulSoup
cafes = {}

cafe_xml_links = [
    "https://europeancoffeetrip.com/cafe-sitemap.xml",
    "https://europeancoffeetrip.com/cafe-sitemap2.xml",
    "https://europeancoffeetrip.com/cafe-sitemap3.xml",
    "https://europeancoffeetrip.com/cafe-sitemap4.xml",
    "https://europeancoffeetrip.com/cafe-sitemap5.xml",
    ]

In [15]:
#! python -m pip install  lxml

In [16]:
for link in cafe_xml_links:
    link_HTML = getHMTLfile_By_Link(link)

    cafeParser = BeautifulSoup(link_HTML, "xml")
    cafe_links_objects = cafeParser.find_all("url")

    for cafe_link in cafe_links_objects:
        loc_tag = cafe_link.find("loc")
        if loc_tag:  # Ensure the <loc> tag exists
            cafes[loc_tag.text] = []

Request to https://europeancoffeetrip.com/cafe-sitemap.xml was successful
Request to https://europeancoffeetrip.com/cafe-sitemap2.xml was successful
Request to https://europeancoffeetrip.com/cafe-sitemap3.xml was successful
Request to https://europeancoffeetrip.com/cafe-sitemap4.xml was successful
Request to https://europeancoffeetrip.com/cafe-sitemap5.xml was successful


In [17]:
len(cafes)

4601

In [18]:
from random import randint
import requests
from bs4 import BeautifulSoup

def rotate_proxy_request(url, proxies):
    
    user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7; rv:85.0) Gecko/20100101 Firefox/85.0"
    ]
    
    proxy = proxies[randint(0, len(proxies)-1)]
    headers = {"User-Agent": user_agents[randint(0, len(user_agents)-1)]}
    
    try:
        response = requests.get(url, proxies={"http": proxy, "https": proxy}, headers=headers, timeout=10)
        
        if response.status_code == 200:
            print(f"Request to {url} was successful with proxy {proxy}")    
            return response.text
        else:
            print(f"Request to {url} failed with proxy {proxy}")
            return None
    except Exception as e:
        print(f"Probably a problem with the used proxy: {proxy}. Proxis left {len(proxies)-1} and {e}")
        proxies.remove(proxy)
        return rotate_proxy_request(url, proxies)

In [ ]:

proxies = ["103.112.144.42:8088",
"190.94.213.58:999",
]


"""cafes = {'https://europeancoffeetrip.com/cafe/kuca-belgrade/': [],
 'https://europeancoffeetrip.com/cafe/waycupkafeterija-belgrade/': []}

for cafe in cafes:
    print(cafe)"""

"cafes = {'https://europeancoffeetrip.com/cafe/kuca-belgrade/': [],\n 'https://europeancoffeetrip.com/cafe/waycupkafeterija-belgrade/': []}\n\nfor cafe in cafes:\n    print(cafe)"

In [20]:
len(cafes)

4601

In [25]:
import json

with open('Cafe-Dict-unique.txt', 'r', encoding='utf-8') as file:
    cafes = json.load(file)

In [26]:
print(cafes)

{'https://europeancoffeetrip.com/cafe/thebookshelfcoffeehouse-tralee/': ['The Bookshelf Tralee', 'Unit 1, Manor Retail and Leisure Park Tralee, Ireland', ['9.674677200', '52.263504000']], 'https://europeancoffeetrip.com/cafe/mustekparallelstation-prague/': [], 'https://europeancoffeetrip.com/cafe/menomoka-modena/': [], 'https://europeancoffeetrip.com/cafe/therustycoffee-budapest/': ['The Rusty Coffee Buda', 'Pasaréti út 51, 1026 Budapest, Hungary', ['19.004516600', '47.516445600']], 'https://europeancoffeetrip.com/cafe/alfabetagama-kosice/': [], 'https://europeancoffeetrip.com/cafe/partisancafeartisanal-paris/': ['Partisan Café Artisanal', '36 R. de Turbigo, 75003 Paris, France', ['2.353093000', '48.864681000']], 'https://europeancoffeetrip.com/cafe/borealcoffeeshopmontblanc-geneva/': [], 'https://europeancoffeetrip.com/cafe/kaffeefabrik-vienna-ottobauer/': [], 'https://europeancoffeetrip.com/cafe/kaffeefabrik-ottobauer-vienna/': [], 'https://europeancoffeetrip.com/cafe/cafse-a-casa-ho

In [27]:
import re
from random import randint
import time

count = 0
for cafe in cafes:
    try:
        if cafes[cafe] == []:
            #address_HTML = rotate_proxy_request(cafe, proxies)
            address_HTML = getHMTLfile_By_Link(cafe)
            if address_HTML != None:
                address_Parser =  BeautifulSoup(address_HTML)
                addresses_link = address_Parser.find_all(
                    "div", 
                    {"class": "cafe-name-address"})
                
                                
                for address_link in addresses_link:
                    name = address_link.find("h1", {"class": "cafe-name"}).text.strip()
                    address = address_link.find("div", {"class": "cafe-address"}).text.strip()
                    #store in dict
                    
                # <div id="cafe-map">                
                coordinates_div = address_Parser.find("div", {"id": "cafe-map"})
                if coordinates_div:
                    img_tag = coordinates_div.find("img", {"class": "lazyload"})
                    if img_tag:
                        src = img_tag.get("data-src")
                        # Use regex to extract latitude and longitude
                        coordinates_match = re.search(r"(\d+\.\d+),(\d+\.\d+)", src)  
                        if coordinates_match:
                            latitude = coordinates_match.group(1)
                            longitude = coordinates_match.group(2)
                            coordinates = [latitude, longitude]  
            
                cafes[cafe] = [name, address, coordinates]
                print(f"Stored: {name}\t{address}\t{coordinates}")
            count += 1
            if count % 100 == 0:
                print(f"Dict: {cafes}")
                
            #to not get detected as a scanner/bruteforcer
            random_sleep = randint(5,10)
            print(f"{random_sleep}=")
            print("\n")
            time.sleep(random_sleep)
        else:
            print(f"Entry for {cafe} already exists")
    except Exception as e:
        print(f"Error with {cafe}: {e}")
        
            

Entry for https://europeancoffeetrip.com/cafe/thebookshelfcoffeehouse-tralee/ already exists
Request to https://europeancoffeetrip.com/cafe/mustekparallelstation-prague/ failed
8=


Request to https://europeancoffeetrip.com/cafe/menomoka-modena/ failed
7=


Entry for https://europeancoffeetrip.com/cafe/therustycoffee-budapest/ already exists
Request to https://europeancoffeetrip.com/cafe/alfabetagama-kosice/ was successful
Error with https://europeancoffeetrip.com/cafe/alfabetagama-kosice/: name 'coordinates' is not defined
Entry for https://europeancoffeetrip.com/cafe/partisancafeartisanal-paris/ already exists
Request to https://europeancoffeetrip.com/cafe/borealcoffeeshopmontblanc-geneva/ was successful
Stored: Boréal Coffee Shop Mont Blanc	Rue du Mont-Blanc 15, CH-1201 Geneva, Switzerland	['6.144028902', '46.208851919']
8=


Request to https://europeancoffeetrip.com/cafe/kaffeefabrik-vienna-ottobauer/ was successful
Stored: 		['6.144028902', '46.208851919']
5=


Request to https://

In [28]:
import json

with open('Cafe-Dict.txt', 'w', encoding='utf-8') as file:
    json.dump(cafes, file, ensure_ascii=False, indent=4)

In [1]:
import json

with open('Cafe-Dict.txt', 'r', encoding='utf-8') as file:
    cafes_from_disk = json.load(file)

In [ ]:
print(cafes_from_disk)

{'https://europeancoffeetrip.com/cafe/thebookshelfcoffeehouse-tralee/': ['The Bookshelf Tralee', 'Unit 1, Manor Retail and Leisure Park Tralee, Ireland', ['9.674677200', '52.263504000']], 'https://europeancoffeetrip.com/cafe/mustekparallelstation-prague/': [], 'https://europeancoffeetrip.com/cafe/menomoka-modena/': [], 'https://europeancoffeetrip.com/cafe/therustycoffee-budapest/': ['The Rusty Coffee Buda', 'Pasaréti út 51, 1026 Budapest, Hungary', ['19.004516600', '47.516445600']], 'https://europeancoffeetrip.com/cafe/alfabetagama-kosice/': [], 'https://europeancoffeetrip.com/cafe/partisancafeartisanal-paris/': ['Partisan Café Artisanal', '36 R. de Turbigo, 75003 Paris, France', ['2.353093000', '48.864681000']], 'https://europeancoffeetrip.com/cafe/borealcoffeeshopmontblanc-geneva/': ['Boréal Coffee Shop Mont Blanc', 'Rue du Mont-Blanc 15, CH-1201 Geneva, Switzerland', ['6.144028902', '46.208851919']], 'https://europeancoffeetrip.com/cafe/kaffeefabrik-vienna-ottobauer/': ['', '', ['6.

#### dict to geojson

In [34]:
cafe_geojson = {
    "type": "FeatureCollection",
    "features": []
}

for url, data in cafes.items():
    if data:
        name, address, coordinates = data
        feature = {
            "type": "Feature",
            "properties": {
                "name": name,
                "description": address
            },
            "geometry": {
                "type": "Point",
                "coordinates": [float(coordinates[0]), float(coordinates[1])]
            }
        }
        cafe_geojson["features"].append(feature)
    

In [35]:
with open('cafe-geojson.json', 'w', encoding='utf-8') as file:
    json.dump(cafe_geojson, file, ensure_ascii=False, indent=4)